# PKIS 1 Dataset creation

In [1]:
%load_ext autoreload
%autoreload 2
import os
import datamol as dm
import pandas as pd
from polaris import curation
import numpy as np
import umap
import re
import seaborn as sns
from matplotlib import pyplot as plt
import seaborn as sns
import os
import pathlib

os.chdir(pathlib.Path("__file__").absolute().parents[2])
from polaris.curation._chemistry_curator import UNIQUE_ID
from polaris.dataset import ColumnAnnotation, Dataset
from utils import load_readme

import warnings

warnings.filterwarnings("ignore")

/Users/lu.zhu/miniconda3/envs/pov3/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/lu.zhu/miniconda3/envs/pov3/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/lu.zhu/miniconda3/envs/pov3/lib/python3.11/site-packages/umap/d

In [2]:
from polaris.utils.types import HubOwner

owner = HubOwner(organizationId="PolarisTest", slug="polaristest")
owner.owner

'PolarisTest'

In [3]:
DATASET_DIR = "gs://polaris-public/datasets/kinases/pkis1_subset_curated"

### Create Dataset

In [4]:
path = "gs://polaris-public/data/Kinases/pkis1_subset_curated_v1.parquet"

table = pd.read_parquet(path)
table.rename(columns={UNIQUE_ID: "UNIQUE_ID"}, inplace=True)

In [5]:
data_cols = [
    "EGFR_(L858R_mutant)",
    "EGFR",
    "KIT_(T6701_mutant)",
    "KIT_(V560G_mutant)",
    "KIT",
    "RET_(V804L_mutant)",
    "RET_(Y791F_mutant)",
    "RET",
]

threshold_dict = {
    "EGFR_(L858R_mutant)": 75,
    "EGFR": 75,
    "KIT_(T6701_mutant)": 75,
    "KIT_(V560G_mutant)": 75,
    "KIT": 75,
    "RET_(V804L_mutant)": 70,
    "RET_(Y791F_mutant)": 70,
    "RET": 70,
}

#### Create annotations for data columns

In [6]:
mutant_anno = {}
mutant_cls_anno = {}

for col in data_cols:
    tar = col.split("_")[0]
    mut_var = re.findall(r"_\((\S+)\)", data_cols[0])
    mut_var = None if len(mut_var) == 0 else mut_var[0]
    if mut_var is None:
        anno = ColumnAnnotation(
            description=f"Percentage of inhibition on {tar} wide type",
            user_attributes={
                "unit": "%",
                "concentration": "1uM",
                "organism": "Human",
                "objective": "Higher value",
                "source": "PKIS1",
                "mutation": None,
                "target": tar,
            },
        )
        cls_anno = ColumnAnnotation(
            protocol=f"Binarized label based on the percentage of inhibition on {tar} wide type",
            user_attributes={
                "thresholds": f"Greather than {threshold_dict[col]}",
                "label_order": "ascending",
                "ref_col": col,
            },
        )
    else:
        anno = ColumnAnnotation(
            description=f"Percentage of inhibition on {tar} with mutation {mut_var}",
            user_attributes={
                "unit": "%",
                "concentration": "1uM",
                "organism": "Human",
                "objective": "Higher value",
                "source": "PKIS1",
                "mutation": mut_var,
                "target": tar,
            },
        )

        cls_anno = ColumnAnnotation(
            description=f"Binarized label based on the percentage of inhibition on {tar} {mut_var}",
            user_attributes={
                "thresholds": f"Greather than {threshold_dict[col]}",
                "label_order": "ascending",
                "ref_col": col,
            },
        )
    mutant_anno[col] = anno
    mutant_cls_anno[f"CLASS_{col}"] = cls_anno

In [7]:
pkis_annotations = {
    # Molecule identifiers
    "UNIQUE_ID": ColumnAnnotation(
        description="Molecular hash ID. See <datamol.mol.hash_mol>"
    ),
    "smiles": ColumnAnnotation(
        description="Molecule SMILES string", modality="molecule"
    ),
    **mutant_anno,
    **mutant_cls_anno,
}

In [8]:
dataset_name = "pkis1_egfr_ret_kit_v1"
dataset = Dataset(
    table=table[pkis_annotations.keys()],
    name=dataset_name,
    description=f"A subset of PKIS dataset only including EGFR, RET, KIT kinases. PKIS is a data set of 367 small-molecule ATP-competitive kinase inhibitors that was screened by the set in activity assays with 224 recombinant kinases and 24 G protein-coupled receptors and in cellular assays of cancer cell proliferation and angiogenesis.",
    source="https://pubmed.ncbi.nlm.nih.gov/26501955",
    annotations=pkis_annotations,
    owner=owner,
    tags=["Kinase", "HitDiscovery", "Selectivity"],
    readme=load_readme(
        "gs://polaris-public/readme/datasets/pkis1_egfr_ret_kit_v1_readme.md"
    ),
)

In [9]:
# save the dataset
SAVE_DIR = f"{DATASET_DIR}/{dataset_name}"
dataset.to_json(SAVE_DIR)

'gs://polaris-public/datasets/kinases/pkis1_subset_curated/pkis1_egfr_ret_kit_v1/dataset.json'